In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm


In [9]:
df = pd.read_csv('../tweet-sentiment-extraction/train.csv')
print(df.columns)
df = pd.read_csv('../tweet-sentiment-extraction/test.csv')
print(df.columns)
df = pd.read_csv('../tweet-sentiment-extraction/sample_submission.csv')
print(df.columns)

Index(['textID', 'text', 'selected_text', 'sentiment'], dtype='object')
Index(['textID', 'text', 'sentiment'], dtype='object')
Index(['textID', 'selected_text'], dtype='object')


In [2]:
def foo(row):
    # print('row:')
    # print(row)
    z = ['none', 'start']
    
    temp = row['start_idx']
    ans = np.zeros(len(row['text'].split()))
    ans[temp] = 1
    ans = np.array(ans, dtype=np.int32)
    ans = ' '.join([str(z[ans_]) for ans_ in ans])
    return ans

In [3]:
def foo2(row):
    try:
        temp = row['start_idx']
        temp += len(row['selected_text'].split())
        temp -= 1
        end_idx = temp
        ans = np.zeros(len(row['text'].split()))
        ans[temp] = 1
    except:
        print(row)
        import sys; sys.exit(0)
    ans = np.array(ans, dtype=np.int32)
    z = ['none', 'end']
    ans = ' '.join([str(z[ans_]) for ans_ in ans])
    return ans

In [4]:
def foo3(row):
    temp = row['text'].find(row['selected_text'])
    if temp>0: 
        if row['text'][temp-1]!=' ': 
            temp = row['text'].rfind(' ',0,temp)
            if temp==-1: 
                temp=0
    temp = len(row['text'][:temp].split())
    return temp

In [5]:
tqdm.pandas()

/home/rushikej/miniconda3/envs/tf_3/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
df = pd.read_csv('../tweet-sentiment-extraction/train.csv')
# print(df.head())
# df = df.head(50)
if len(df[df['textID']=='fdb77c3752'])!=0:
    df = df.drop([314])

df['start_idx'] = df.progress_apply(foo3, axis=1)
df['start'] = df.progress_apply(foo, axis=1)
df['end'] = df.progress_apply(foo2, axis=1)
df.head()

100%|██████████| 27480/27480 [00:01<00:00, 19135.27it/s]


,textID,text,selected_text,sentiment,start_idx,start,end
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0,start none none none none none none,none none none none none none end
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,0,start none none none none none none none none ...,none end none none none none none none none none
2,088c60f138,my boss is bullying me...,bullying me,negative,3,none none none start none,none none none none end
3,9642c003ef,what interview! leave me alone,leave me alone,negative,2,none none start none none,none none none none end
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,0,start none none none none none none none none ...,none none end none none none none none none no...


In [7]:
# def foo4(df):
#     df['t_text'] = df['text'].apply(lambda x: tokenizer.tokenize(str(x)))
#     df['t_selected_text'] = df['selected_text'].apply(lambda x: tokenizer.tokenize(str(x)))
#     def func(row):
#         x,y = row['t_text'],row['t_selected_text'][:]
#         for offset in range(len(x)):
#             d = dict(zip(x[offset:],y))
#             #when k = v that means we found the offset
#             check = [k==v for k,v in d.items()]
#             if all(check)== True:
#                 break 
#         return [0]*offset + [1]*len(y) + [0]* (len(x)-offset-len(y))
#     df['targets'] = df.apply(func,axis=1)
#     return df

In [8]:
# df_ = foo4(df)

In [9]:
# def foo4(row):
#     x,y = row['t_text'],row['t_selected_text'][:]
#     for offset in range(len(x)):
#         d = dict(zip(x[offset:],y))
#         #when k = v that means we found the offset
#         check = [k==v for k,v in d.items()]
#         if all(check)== True:
#             break 
#     return [0]*offset + [1]*len(y) + [0]* (len(x)-offset-len(y))

In [10]:
# df.iloc[0]

In [11]:
# foo4(df.iloc[0])

In [12]:
# df.isna

In [13]:
# df['end'].isnull().all()==False

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import time
import random

In [18]:
# train_df.iloc[4]['start']

In [19]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1)

In [20]:
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [21]:
train_df.to_csv('df_changed_train.csv')
val_df.to_csv('df_changed_val.csv')
test_df.to_csv('df_changed_test.csv')

In [22]:
train_path = 'df_changed_train.csv'
test_path = 'df_changed_test.csv'
val_path = 'df_changed_val.csv'

In [23]:
# data.Field?

In [24]:
# import spacy
# spacy_en = spacy.load('en')

# def tokenizer(text): # create a tokenizer function
#     return [tok.text for tok in spacy_en.tokenizer(text)]

In [25]:
# data.Field?

In [26]:
text_field = data.Field(sequential=True)
# text_id_field = data.Field(sequential=False)
# selected_text_field = data.Field(sequential=True)
labels_field = data.Field(sequential=False)
start_field = data.Field(unk_token=None, pad_token='none')
end_field = data.Field(unk_token=None, pad_token='none')

In [27]:
# start_field.vocab.freqs

In [28]:
# data.Field?

In [29]:
fields={    # 'textID': ('textID', text_id_field),
            'text': ('text', text_field),
            # 'selected_text': ('selected_text', selected_text_field),
            'sentiment': ('labels', labels_field), 
            'start': ('start', start_field), 
            'end': ('end', end_field)
        }

In [30]:
# data.Field?

In [31]:
train_data, val_data, test_data = data.TabularDataset.splits(
    path='', 
    train=train_path,
    validation=val_path, 
    test=test_path, 
    format='csv',
    fields=fields)

In [32]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(val_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 22258
Number of validation examples: 2474
Number of testing examples: 2748


In [33]:
text_field.build_vocab(train_data, 
                 min_freq = 5,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)
labels_field.build_vocab(train_data)
start_field.build_vocab(train_data)
end_field.build_vocab(train_data)

In [34]:
start_field.vocab

In [35]:
BATCH_SIZE = 32

device = torch.device('cuda:2')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [87]:
# def jaccard_loss(pred, target, smooth=1e-10):
#     I = (torch.sigmoid(pred) * target).sum(axis=1, keepdim=True)
#     P = pred.sum(axis=1, keepdim=True)
#     T = target.sum(axis=1, keepdim=True)
#     loss = 1 - ((I + smooth) / (P + T - I + smooth))
#     return loss

In [223]:
class SentimentExtractor(nn.Module):
    def __init__(self,
                input_dim,
                outputs_dim,
                embedding_dim,
                hidden_dim,
                pad_idx,
                dropout_rate,
                num_layers,
                bidirectional
                ):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        
        if num_layers == 1:
            dropout_rate=0
            
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 
                            num_layers=num_layers, bidirectional=bidirectional, dropout=dropout_rate)
        
        # self.dropout = nn.Dropout(dropout_rate)
        
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, outputs_dim)
        else:
            self.fc = nn.Linear(hidden_dim, outputs_dim)
    
    def forward(self, text):
        # embeddings = self.dropout(self.embedding(text))
        embeddings = self.embedding(text)
        output, (hidden, cell) = self.lstm(embeddings)
        # return self.fc(self.dropout(output))
        return self.fc(output), self.fc(output)

In [224]:
input_dim = len(text_field.vocab)
outputs_dim = 1
hidden_dim = 64
num_layers = 1
bidirectional = False
dropout_rate = 0.25
pad_idx = text_field.vocab.stoi[text_field.pad_token]
embedding_dim = 100

model = SentimentExtractor(input_dim=input_dim,
                 outputs_dim=outputs_dim,
                 hidden_dim=hidden_dim,
                 num_layers=num_layers,
                 bidirectional=bidirectional,
                 dropout_rate=dropout_rate,
                 embedding_dim=embedding_dim,
                 pad_idx=pad_idx)

# tag_pad_idx = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=tag_pad_idx)

In [225]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights);

In [226]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model):,} trainable parameters')

In [227]:
model.to(device)
# criterion.to(device_cpu)
for param in model.parameters():
    param.requires_grad = True

pretrained_embeddings = text_field.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings);

model.embedding.weight.data[1] = torch.zeros(embedding_dim)
optimizer = optim.Adam(model.parameters())

In [228]:
def custom_categorical_accuracy(y_pred, y_true, tag_pad_idx):
    y_pred = y_pred.argmax(dim=1, keepdim=True)
#     print(y_true)
#     print(tag_pad_idx)
#     print(y_true!=tag_pad_idx)
    non_pad_element_idxs = (y_true!=tag_pad_idx).nonzero()
    correct = y_pred[non_pad_element_idxs].squeeze(1).eq(y_true[non_pad_element_idxs])
    return correct.sum() / torch.FloatTensor([y_true[non_pad_element_idxs].shape[0]])

In [229]:
model.to(device);
# my_loss.to(device)

In [242]:
# torch.tensor?

In [1]:
def my_loss(true, pred):
    # this is the new criterion
    
    # expected true to be of shape (batch_size, no_of_toks) -- but only 0, 1
    # expecting pred to be of shape (batch_size, no_of_toks) -- but float values
    
    true_start, true_end = true
    pred_start, pred_end = pred
    
    pred_start_idx = torch.argmax(pred_start, dim=1, keepdim=False)
    pred_end_idx = torch.argmax(pred_end, dim=1, keepdim=False)
    # print(pred_start_idx.cpu().numpy())
    # print(pred_end_idx.cpu().numpy())
    # TODO: fix this later
    true_start_idx = torch.tensor([temp[1] for temp in (true_start==1).nonzero()], requires_grad=True).to(device)
    true_end_idx = torch.tensor([temp[1] for temp in (true_end==1).nonzero()], requires_grad=True).to(device)
    # print(true_start_idx.cpu().numpy())
    # print(true_end_idx.cpu().numpy())

    I = torch.min(true_end_idx, pred_end_idx) - torch.max(true_start_idx, pred_start_idx)
    # print(I.cpu().numpy())
    P = pred_end_idx - pred_start_idx
    T = true_end_idx - true_start_idx
    
    smooth = 1e-6
    
    loss = 1 - ((I + smooth) / (P + T - I + smooth))
    
    loss[(I<0).nonzero()] = 1
    loss[(pred_end_idx<pred_start_idx).nonzero()] = 10
    
    return loss

In [244]:
with torch.no_grad():
    model.eval()
    for batch in train_iterator:
        text = batch.text
    #     tags = batch.udtags
        print('Text shape: ', text.shape)
        start = batch.start
        print(start.shape)
        end = batch.end
        start = torch.transpose(start, 0, 1).to(device)
        print('Start shape: ', start.shape)
        end = torch.transpose(end, 0, 1).to(device)
        print('End shape: ', end.shape)
        preds_start, preds_end = model(batch.text)
    #     print(batch.end.shape)
        # print(type(text[0][0]))
        # print(text[0][0].cpu().data.numpy())
        # print(text)
        # for text_ in text.T:
        #     pprint(text_)
        #     print()
    #     print(tags.shape)
    #     predictions = model(text)
    #     print(predictions.shape)
    #     predictions = predictions.view(-1, predictions.shape[-1])
    #     print(predictions.shape)
    #     print(tags.shape)
    #     tags = tags.view(-1)
    #     print(tags.shape)
        # print()
    #     print(predictions.shape)
        # print(tags)
        
        preds_start, preds_end = model(batch.text)
        preds_start = torch.transpose(preds_start.view(preds_start.shape[0], -1), 0, 1)
        print('Preds_start shape: ', preds_start.shape)
        
        preds_end = torch.transpose(preds_end.view(preds_end.shape[0], -1), 0, 1)
        print('Preds_end shape: ', preds_end.shape)
        
        j = my_loss((start, end), (preds_start,preds_end))
        print('j shape: ', j.shape)
#         print(j)
#         print(preds_start)
#         preds_start = torch.transpose(preds_start,0,1)
#         preds_end = torch.transpose()
        
        
    #     print(batch.end.shape)
        # print(type(text[0][0]))
        # print(text[0][0].cpu().data.numpy())
        # print(text)
        # for text_ in text.T:
        #     pprint(text_)
        #     print()
    #     print(tags.shape)
    #     predictions = model(text)
    #     print(predictions.shape)
    #     predictions = predictions.view(-1, predictions.shape[-1])
    #     print(predictions.shape)
    #     print(tags.shape)
    #     tags = tags.view(-1)
    #     print(tags.shape)
        # print()
    #     print(predictions.shape)
        # print(tags)
        break

Text shape:  torch.Size([28, 32])
torch.Size([28, 32])
Start shape:  torch.Size([32, 28])
End shape:  torch.Size([32, 28])
Preds_start shape:  torch.Size([32, 28])
Preds_end shape:  torch.Size([32, 28])
j shape:  torch.Size([32])


In [38]:
# # torch.seed()
# z = torch.randn(2,1)
# z2 = torch.randn(2,1)

In [187]:
# criterion(z,z2)

In [188]:
# criterion(torch.FloatTensor(1), torch.FloatTensor(2))

In [189]:
# for batch in train_iterator:
#     text = batch.text
#     tags = batch.udtags

#     y_pred = model(text)
#     print(y_pred)
#     y_pred = y_pred.view(-1, y_pred.shape[-1])

#     tags = tags.view(-1)

#     loss = criterion(y_pred, tags)


In [263]:
def train_one_epoch(model, iterator, loss_func, optimizer):
    model.train()
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    optimizer.zero_grad()
    with torch.enable_grad():
        for batch in train_iterator:
            text = batch.text
    #         print('Text shape: ', text.shape)
            start = batch.start
    #         print(start.shape)
            end = batch.end
            start = torch.transpose(start, 0, 1).to(device)
    #         print('Start shape: ', start.shape)
            end = torch.transpose(end, 0, 1).to(device)
    #         print('End shape: ', end.shape)
            preds_start, preds_end = model(batch.text)

    #         preds_start, preds_end = model(batch.text)
            preds_start = torch.transpose(preds_start.view(preds_start.shape[0], -1), 0, 1)
    #         print('Preds_start shape: ', preds_start.shape)

            preds_end = torch.transpose(preds_end.view(preds_end.shape[0], -1), 0, 1)
    #         print('Preds_end shape: ', preds_end.shape)
#             preds_start.require_grad=True
#             preds_end.require_grad = True
            j_loss = loss_func((start, end), (preds_start,preds_end))
    #         print('j shape: ', j.shape)
            # acc = custom_categorical_accuracy(y_pred, tags, tag_pad_idx)
            print(j_loss.shape)
            
            j_loss.backward()
            optimizer.step()

            epoch_loss += j_loss.item()
        
    return epoch_loss/len(iterator)

In [264]:
def val_one_epoch(model, iterator, loss_func):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        
        for batch in iterator:
            text = batch.text
            tags = batch.udtags

            y_pred = model(text)
            y_pred = y_pred.view(-1, y_pred.shape[-1])

            tags = tags.view(-1)
            

            loss = loss_func(y_pred, tags)

            epoch_loss += loss.item()

    return epoch_loss/len(iterator)

In [265]:
# device_cpu = torch.device('cpu')

In [266]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [267]:
# for batch in train_iterator:
#     pass

In [268]:
# vars(train_iterator.data()[0])

In [269]:
# for e in train_iterator.data():
# #     print(e.text)
#     try:
#         for udtag in e.udtags:
#             z = (UD_TAGS.vocab.stoi[udtag])
#     except Exception as e:
#         print(e)
#         print(vars(e))
# #     break

In [270]:
# model.to(torch.device('cpu'))

In [271]:

n_epochs = 10

best_val_loss = float('inf')

for epoch in range(n_epochs):
    
    start_time = time.time()
    
    train_loss, train_acc = train_one_epoch(model, train_iterator, my_loss, optimizer)
    val_loss, val_acc = val_one_epoch(model, valid_iterator, my_loss)
    
    end_time = time.time()
    
    epoch_time_min, epoch_time_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_time_min}m {epoch_time_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
    

torch.Size([32])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [47]:
# x = torch.randn(100,1000)
# y = torch.randn(1000,1000)

In [55]:
# device = torch.device('cpu')

In [50]:
# x.to(device); y.to(device)

In [51]:
# z = torch.matmul(x,y)

In [155]:
# x = torch.randn((32,18))

In [154]:
# x.nonzero().shape

In [52]:
# loss = nn.CrossEntropyLoss()
# input_ = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# print(input_); print(target)
# output = loss(input_, target)

In [ ]:
def my_loss(true, pred):
    # this is the new criterion
    
    # expected true to be of shape (batch_size, None) -- but only 0, 1
    # expecting pred to be of shape (batch_size, None) -- but float values
    
    true_start, true_end = true
    pred_start, pred_end = pred
    
    pred_start_idx = (pred_start==1).nonzero()
    pred_end_idx = (pred_end==1).nonzero()
    
    if pred_start_idx > pred_end_idx: return 100
    
    true_start_idx = (true_start==1).nonzero()
    true_end_idx = (true_end==1).nonzero()
    
    I = torch.min(true_end_idx, pred_end_idx) - torch.max(true_start_idx, pred_start_idx)
    if I < 0: return 1
    P = pred_end_idx - pred_start_idx
    T = true_end_idx - true_start_idx
    
    smooth = 1e-6
    
    loss = 1 - ((I + smooth) / (P + T - I + smooth))
    return loss

In [297]:
from torch.autograd import Variable
import torch.nn.functional as F

In [298]:
def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * Variable(signs))
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.relu(errors_sorted), Variable(grad))
    return loss

In [299]:
x1 = torch.FloatTensor(2,3)
a = [[1,0,1],[0,1,1]]
a = np.array([np.array(aa,dtype=np.float32) for aa in a])
x2 = torch.tensor(a)

In [302]:
x1 = x1.view(-1)
x2 = x2.view(-1)

In [303]:
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.float().cumsum(0)
    union = gts + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


In [304]:
lovasz_hinge_flat(x1, x2)

tensor(23.5030)